## Illustration of **wandb** for visual inspection og ML optimization and validation

- Sign-up at https://wandb.ai/ 
- Install wandb in the same virtual environment as tensorflow
  - conda install -c conda-forge wandb
- in the prompt: wandb login
  - copy the key from the web page and paste it in the prompt

In [ ]:
import os
import wandb

from wandb.keras import WandbCallback

In [ ]:
import tensorflow as tf

from matplotlib import pyplot as plt

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = x_train / 255
x_test = x_test / 255

In [ ]:
def build_model(size, activation, weight_regularizer, dropout, batch_norm):
    layers = [tf.keras.layers.Flatten(input_shape=(28, 28))]
    
    nb_layers = size
    nb_neurons = size ** 2 * 4
    
    for i in range(nb_layers):
        layers.append(tf.keras.layers.Dense(nb_neurons, activation=activation, 
                                            kernel_regularizer=weight_regularizer))
        if batch_norm:
            layers.append(tf.keras.layers.BatchNormalization())
        if dropout:
            layers.append(tf.keras.layers.Dropout(0.2))

    layers.append(tf.keras.layers.Dense(10, activation='softmax'))
    
    model = tf.keras.models.Sequential(layers)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.0001),
        metrics=['sparse_categorical_crossentropy', 'accuracy'],
        )
    
    return model

In [ ]:
model_small = build_model(1, 'relu', None, False, False)
model_small.summary()

In [ ]:
## Here you need to supply your own wandb API key (if it has not been set by wandb login at prompt has not been used), project name and entity name.

#os.environ["WANDB_API_KEY"]= 'YOUR_API_KEY'
os.environ["WANDB_NAME"]= 'fashion-mnist-small'
os.environ["WANDB_NOTEBOOK_NAME"]='TryWandb.ipynb'
run = wandb.init(project="amlfall23",entity='christianmdahl',job_type='train')
config = wandb.config

In [ ]:
# Define WandbCallback for experiment tracking
wandb_callback = WandbCallback(monitor='val_loss',verbose=True,save_model=False,
                               log_weights=True,
                               log_evaluation=True,
                               validation_steps=2)
callbacks = [wandb_callback]


In [ ]:
history_small = model_small.fit(x_train, y_train,
                                validation_data=(x_test, y_test),
                                epochs=40, verbose=1,
                                callbacks = callbacks)